# Netflix Data Analysis

Data downloaded from Kaggle web site:  https://www.kaggle.com/shivamb/netflix-shows?select=netflix_titles.csv

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
netflix_df = pd.read_csv("netflix_titles.csv")
netflix_df.head(10)

In [ ]:
# Check data types of columns
netflix_df.dtypes

In [ ]:
netflix_df.describe()

In [ ]:
# New dataframe with split value columns to change column type
# See: https://pandas.pydata.org/pandas-docs/version/0.23.1/generated/pandas.Series.str.split.html
column_min = netflix_df["duration"].str.split(" ", n = 1, expand = True)

In [ ]:
netflix_df["duration (min)"]= column_min[0] #Add a column to the dataframe

# Change column type from object to int64
netflix_df['duration (min)'] = pd.to_numeric(netflix_df['duration (min)'])

# Check data type of the column
netflix_df.dtypes

In [ ]:
netflix_df.shape

In [ ]:
netflix_df.describe()

In [ ]:
# Filter the data so that only those sightings in the movies are in a DataFrame
movie_based_df = netflix_df[netflix_df["type"] == "Movie"] #Like WHERE condition in SQL
movie_based_df.head(3)

In [ ]:
movie_based_df.describe()

In [ ]:
movie_based_df = movie_based_df[movie_based_df["duration (min)"] > 60] #Like WHERE condition in SQL

In [ ]:
movie_based_df.describe()

In [ ]:
# Generate a pie plot showing the distribution of Movies vs TV Shows using pandas
production_type = netflix_df.groupby('type')
# create a DataFrame
production_type_count = pd.DataFrame(production_type['type'].count())
production_type_count

In [ ]:
fig, ax = plt.subplots()

production_type_count.plot(kind='pie', y="type", autopct='%1.1f%%', figsize=(10,8),shadow=True, startangle=140, legend=False, fontsize=12, ax=ax)

ax.set_ylabel('Production Type')
ax.set_title("Production Type Distribution",fontsize=15)

In [ ]:
# Ratings data
ratings_count = netflix_df.groupby("rating")

ratings_count_df = pd.DataFrame(ratings_count["rating"].count())
ratings_count_df

In [ ]:
# create bar chart showing Ratings content added over the years - Overall
fig, ax = plt.subplots()
ratings_count_df.plot(kind='bar',figsize=(10,8),legend=False, ax=ax)
ax.set_xlabel("Ratings",fontsize=12)
ax.set_ylabel("Count",fontsize=12)
ax.set_title("Ratings Distribution",fontsize=15)
#plt.savefig("figures/RatingsDist_Bar.png")

# Multiple data sources

In [ ]:
Newfile_df=pd.DataFrame(netflix_df)
Newfile_df

### Reading additional dataset: Female Director List from IMDB

In [ ]:
# Reading the IMDB Female Directors List (Additional Datset)(NRB):

IMDBFemaleDir = pd.read_csv("LIST OF FEMALE DIRECTORS.csv", encoding = 'unicode_escape')
IMDBFemaleDir.head()

In [ ]:
# Updating the column heading (NRB):

IMDBFemaleDir_df = IMDBFemaleDir.rename(columns={"Name": "director"})
IMDBFemaleDir_df

In [ ]:
# Adding the Gender Column(NRB):

IMDBFemaleDir_df["Gender"] = "Female" #Add a new column
IMDBFemaleDir_df

In [ ]:
# Dropping off all the unnecessary columns from IMDB Dataset (NRB):

Updated_IMDBFemaleDir_df = IMDBFemaleDir_df.drop(['Position', 'Const', 'Created','Modified','Description',
                                                 'Known For','Birth Date'], axis=1)
Updated_IMDBFemaleDir_df

In [ ]:
# Dropping off Duplicates (NRB):

Updated_IMDBFemaleDir_df_nodups = Updated_IMDBFemaleDir_df.drop_duplicates()
Updated_IMDBFemaleDir_df_nodups

In [ ]:
# Merging both files for analysis (NRB)
# Merge = JOIN in relational db
Combined_files1= pd.merge(Newfile_df,Updated_IMDBFemaleDir_df_nodups,how='left',on=['director'],sort=False)
Combined_files1

In [ ]:
check_female_df = Combined_files1[Combined_files1["Gender"] == "Female"]
check_female_df

In [ ]:
# Filling value as Other in column Gender for directors other than Female:(NRB)
Combined_files1["Gender"].fillna("Other", inplace=True)
Combined_files1

In [ ]:
# Creating new DataFrame to show the ratio of Female Directors versus Other 
# who directed Movies and dropping off the TV Show values:(NRB)

df2 = Combined_files1.groupby(['type','Gender']).count()['director']
df3 = pd.DataFrame(df2.drop('TV Show'))
df3

In [ ]:
# Plotting the dataframe:(NRB)

plt.figure(figsize=(6,6))
labels=["Female Directors", "Other Directors"]
wedgeprops = {"linewidth": 4, 'width':1}
explode = (0.3,0)
plt.pie(df3, labels=labels,explode=explode,autopct="%1.3f%%", shadow=True, startangle=45, 
        textprops={'fontsize': 12}, wedgeprops = wedgeprops,)
plt.xlabel('Directors', fontsize=12)
plt.tight_layout()
plt.suptitle("Movie Directors: Other vs Female", fontsize=15)
plt.show()

In [ ]:
# Creating dataframe of data grouped by gender and duration (NRB)

df4 = Combined_files1[Combined_files1['type'] == 'Movie']
df5 = pd.DataFrame(df4.groupby(['Gender','duration (min)']).size().reset_index(name="Count"))

In [ ]:
# Dropping off the 'Other' from Gender to analyze the relationship between 
# the Female Directors and the duration of movies they produced: (NRB)

df6 = df5[df5.Gender != "Other"].sort_values(by='duration (min)',ascending=False)
df6

In [ ]:
df6[['duration (min)']].plot(kind='hist',bins= [0,70,90,110,130,150,170,190],rwidth=0.8)

In [ ]:
# Creating bins for duration(mins) of Movies and the assigning the labels
# Creating a new column for the labels - FeatureFilmLength (NRB)

bins= [0,70,90,110,130,150,170,190]
feature_length= ["(< 70mins)", "(70-89mins)", "(90-109mins)", 
                 "(110-129mins)","(130-149mins)","(150-169mins)",
                 "(170-189mins)"]

df6["FeatureFilmLength"]= pd.cut(df6["duration (min)"],bins=bins,labels=feature_length)
df6